<a href="https://colab.research.google.com/github/Samgomes2510/dashborad-ecommerce/blob/main/3_Particoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
from google.colab import files
files.upload()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
produtos = spark.read.csv("produtos.csv", header=True, inferSchema=True)
vendedores = spark.read.csv("vendedores.csv", header=True, inferSchema=True)
clientes = spark.read.csv("clientes.csv", header=True, inferSchema=True)
itens_pedido = spark.read.csv("itens_pedido.csv", header=True, inferSchema=True)
pagamentos_pedido = spark.read.csv("pagamentos_pedido.csv", header=True, inferSchema=True)
avaliacoes_pedido = spark.read.csv("avaliacoes_pedido.csv", header=True, inferSchema=True)
pedidos = spark.read.csv("pedidos.csv", header=True, inferSchema=True)

In [ ]:
produtos.show()

In [ ]:
rdd = spark.sparkContext.parallelize(range(100))
print("Numero padrão de particões:", rdd.getNumPartitions())

In [ ]:
rdd = spark.sparkContext.parallelize(range(100), 10)
print("Numero padrão de particões após modificação:", rdd.getNumPartitions())

In [ ]:
reduced_rdd = rdd.coalesce(10)
print("Numero padrão de particões após coalesce:", reduced_rdd.getNumPartitions())

In [ ]:
repartitioned_rdd = reduced_rdd.repartition(20)
print("Numero padrão de particões após repartition:", repartitioned_rdd.getNumPartitions())

In [ ]:
print("Numero de partições padrão de produtos:", produtos.rdd.getNumPartitions())

produtos = spark.read.csv("produtos.csv", header=True, inferSchema=True).repartition(5)

print("Numero de partições após repartition de produtos:", produtos.rdd.getNumPartitions())

In [ ]:
produtos.groupBy("categoria_produto").count().show()
print(produtos.select("categoria_produto").distinct().count())

produtos_part_categoria = produtos.repartition("categoria_produto")
print(produtos_part_categoria.rdd.getNumPartitions())

In [ ]:
produtos_part_categoria = produtos.repartition(8, "categoria_produto")
print(produtos_part_categoria.rdd.getNumPartitions())

particoes = produtos_part_categoria.rdd.glom().map(len).collect()
print("Numero de registro por particoes:", particoes)

In [ ]:
produtos.write.mode("overwrite").csv("produtos_default_parquet")

In [ ]:
produtos.coalesce(1).write.mode("overwrite").parquet("produtos_unicos_arquivo_parquet")

In [ ]:
produtos.repartition(10).write.mode("overwrite").parquet("produtos_part10_parquet")

In [ ]:
produtos.write.mode("overwrite").partitionBy("categoria_produto").parquet("produtos_part_categoria_parquet")

In [ ]:
produtos.coalesce(1).write.mode("overwrite").partitionBy("categoria_produto").parquet("produtos_unico_part_categoria_parquet")

In [ ]:
spark.stop()